In [59]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# 4.1 FewShotPromptTemplate

In [2]:
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [3]:
t = PromptTemplate.from_template("What is the capital of {country}?")
t

PromptTemplate(input_variables=['country'], template='What is the capital of {country}?')

In [4]:
t.format(country="Spain")

'What is the capital of Spain?'

In [5]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

In [6]:
chat.predict("What do you know about Greece?")

Greece is a country located in southeastern Europe, known for its rich history, cultural heritage, and stunning landscapes. Here are some key points about Greece:

1. **Geography**: Greece is situated on the southern tip of the Balkan Peninsula and is bordered by Albania, North Macedonia, Bulgaria, and Turkey. It has a vast coastline along the Aegean, Ionian, and Mediterranean Seas, and is made up of numerous islands, with Crete being the largest.

2. **History**: Greece is often referred to as the cradle of Western civilization. It was the birthplace of democracy, philosophy, theater, and the Olympic Games. Ancient Greece was home to famous city-states like Athens and Sparta, and notable figures such as Socrates, Plato, and Aristotle.

3. **Culture**: Greek culture is characterized by its contributions to art, literature, and science. The country has a rich tradition of mythology, with gods and goddesses like Zeus, Athena, and Apollo. Greek cuisine is also famous, featuring dishes lik

"Greece is a country located in southeastern Europe, known for its rich history, cultural heritage, and stunning landscapes. Here are some key points about Greece:\n\n1. **Geography**: Greece is situated on the southern tip of the Balkan Peninsula and is bordered by Albania, North Macedonia, Bulgaria, and Turkey. It has a vast coastline along the Aegean, Ionian, and Mediterranean Seas, and is made up of numerous islands, with Crete being the largest.\n\n2. **History**: Greece is often referred to as the cradle of Western civilization. It was the birthplace of democracy, philosophy, theater, and the Olympic Games. Ancient Greece was home to famous city-states like Athens and Sparta, and notable figures such as Socrates, Plato, and Aristotle.\n\n3. **Culture**: Greek culture is characterized by its contributions to art, literature, and science. The country has a rich tradition of mythology, with gods and goddesses like Zeus, Athena, and Apollo. Greek cuisine is also famous, featuring dis

In [7]:
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

In [8]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt

FewShotPromptTemplate(input_variables=['country'], examples=[{'question': 'What do you know about France?', 'answer': '\n            Here is what I know:\n            Capital: Paris\n            Language: French\n            Food: Wine and Cheese\n            Currency: Euro\n            '}, {'question': 'What do you know about Italy?', 'answer': '\n            I know this:\n            Capital: Rome\n            Language: Italian\n            Food: Pizza and Pasta\n            Currency: Euro\n            '}, {'question': 'What do you know about Greece?', 'answer': '\n            I know this:\n            Capital: Athens\n            Language: Greek\n            Food: Souvlaki and Feta Cheese\n            Currency: Euro\n            '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], template='\n    Human: {question}\n    AI: {answer}\n'), suffix='Human: What do you know about {country}?')

In [9]:
prompt.format(country="Germany")

'\n    Human: What do you know about France?\n    AI: \n            Here is what I know:\n            Capital: Paris\n            Language: French\n            Food: Wine and Cheese\n            Currency: Euro\n            \n\n\n\n    Human: What do you know about Italy?\n    AI: \n            I know this:\n            Capital: Rome\n            Language: Italian\n            Food: Pizza and Pasta\n            Currency: Euro\n            \n\n\n\n    Human: What do you know about Greece?\n    AI: \n            I know this:\n            Capital: Athens\n            Language: Greek\n            Food: Souvlaki and Feta Cheese\n            Currency: Euro\n            \n\n\nHuman: What do you know about Germany?'

In [10]:
chain = prompt | chat

chain.invoke({"country": "Germnay"})

AI: 
            I know this:
            Capital: Berlin
            Language: German
            Food: Sausages and Sauerkraut
            Currency: Euro

AIMessageChunk(content='AI: \n            I know this:\n            Capital: Berlin\n            Language: German\n            Food: Sausages and Sauerkraut\n            Currency: Euro')

# 4.2 FewShowChatMessagePromptTemplate

In [11]:
examples = [
    {
        "country": "France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
    },
    {
        "country": "Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
    },
    {
        "country": "Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

fs_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. Youu give short answers."),
        fs_example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Republic of Korea"})


            I know this:
            Capital: Seoul
            Language: Korean
            Food: Kimchi and Bibimbap
            Currency: South Korean Won (KRW)
            

AIMessageChunk(content='\n            I know this:\n            Capital: Seoul\n            Language: Korean\n            Food: Kimchi and Bibimbap\n            Currency: South Korean Won (KRW)\n            ')

# 4.3 LengthBasedExampleSelector

In [12]:
from typing import Any, Dict


class RandomExapleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        return super().add_example(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [13]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     examples=examples, example_prompt=example_prompt, max_length=120
# )

example_selector = RandomExapleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert"),
#     fs_example_prompt,
#     ("human", "What do you know about {country}?")
# ])


# chain = final_prompt | chat

# chain.invoke({"country": "Republic of Korea"})

'\n    Human: What do you know about Greece?\n    AI: \n            I know this:\n            Capital: Athens\n            Language: Greek\n            Food: Souvlaki and Feta Cheese\n            Currency: Euro\n            \n\n\nHuman: What do you know about Brazil?'

# 4.4 Serialization and Composition

In [14]:
# load prompt in json format.

prompt_json = load_prompt("./prompt.json")

prompt_json.format(country="South Korea")

'What is the capital of South Korea'

In [15]:
# load prompt in yaml format.

prompt_yaml = load_prompt("./prompt.json")

prompt_yaml.format(country="France")

'What is the capital of France'

In [16]:
# pipeline

intro = PromptTemplate.from_template(
    """
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
    """
{intro}

{example}

{start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

full_prompt.format(
    character="Assassin",
    example_question="What is your location",
    example_answer="Shhhh, That is secret",
    question="Who is your target?",
)

'\n\nYou are a role playing assistant.\nAnd you are impersonating a Assassin\n\n\n\nThis is an example of how you talk:\n\nHuman: What is your location\nYou: Shhhh, That is secret\n\n\n\nStart now!\n\nHuman: Who is your target?\nYou:\n\n'

In [17]:
chain = full_prompt | chat

chain.invoke(
    {
        "character": "Assassin",
        "example_question": "What is your location",
        "example_answer": "Shhhh, That is secret",
        "question": "Who is your target?",
    }
)

Shhhh, that information is classified.

AIMessageChunk(content='Shhhh, that information is classified.')

# 4.5 Caching

In [39]:
# set_llm_cache(InMemoryCache())
# set_debug(True)

In [40]:
import time

In [41]:
chat = ChatOpenAI(
    model="gpt-4o-mini",
)

In [42]:
chat.predict("How do you make italian pasta ")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [8.27s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe to guide you through making fresh pasta dough, which can be used for various pasta shapes:\n\n### Ingredients\n\n- 2 cups (250g) all-purpose flour or \"00\" flour (for a finer texture)\n- 3 large eggs\n- A pinch of salt\n- Extra flour for dusting\n\n### Instructions\n\n1. **Make the Dough:**\n   - On a clean surface or in a large mixing bowl, create a mound with the flour. Make a well in the center.\n   - Crack the eggs into the well and add a pinch of salt.\n   - Using a fork, beat the eggs gently and gradually incorporate the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n2. **Knead the Dough:**\n   - Gather the dough into a 

'Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe to guide you through making fresh pasta dough, which can be used for various pasta shapes:\n\n### Ingredients\n\n- 2 cups (250g) all-purpose flour or "00" flour (for a finer texture)\n- 3 large eggs\n- A pinch of salt\n- Extra flour for dusting\n\n### Instructions\n\n1. **Make the Dough:**\n   - On a clean surface or in a large mixing bowl, create a mound with the flour. Make a well in the center.\n   - Crack the eggs into the well and add a pinch of salt.\n   - Using a fork, beat the eggs gently and gradually incorporate the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n2. **Knead the Dough:**\n   - Gather the dough into a ball and begin kneading it with your hands. Knead for about 8-10 minutes until the dough is smooth and elastic. If the dough is too sticky, sprinkle a little more flour as needed.\n   - Wrap the dough in plastic wrap or cover it with a cloth and

In [43]:
chat.predict("How do you make italian pasta ")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe to guide you through making fresh pasta dough, which can be used for various pasta shapes:\n\n### Ingredients\n\n- 2 cups (250g) all-purpose flour or \"00\" flour (for a finer texture)\n- 3 large eggs\n- A pinch of salt\n- Extra flour for dusting\n\n### Instructions\n\n1. **Make the Dough:**\n   - On a clean surface or in a large mixing bowl, create a mound with the flour. Make a well in the center.\n   - Crack the eggs into the well and add a pinch of salt.\n   - Using a fork, beat the eggs gently and gradually incorporate the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n2. **Knead the Dough:**\n   - Gather the dough into a ball 

'Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe to guide you through making fresh pasta dough, which can be used for various pasta shapes:\n\n### Ingredients\n\n- 2 cups (250g) all-purpose flour or "00" flour (for a finer texture)\n- 3 large eggs\n- A pinch of salt\n- Extra flour for dusting\n\n### Instructions\n\n1. **Make the Dough:**\n   - On a clean surface or in a large mixing bowl, create a mound with the flour. Make a well in the center.\n   - Crack the eggs into the well and add a pinch of salt.\n   - Using a fork, beat the eggs gently and gradually incorporate the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n2. **Knead the Dough:**\n   - Gather the dough into a ball and begin kneading it with your hands. Knead for about 8-10 minutes until the dough is smooth and elastic. If the dough is too sticky, sprinkle a little more flour as needed.\n   - Wrap the dough in plastic wrap or cover it with a cloth and

In [45]:
# Caching in DB

set_llm_cache(SQLiteCache("cache.db"))

chat.predict("How do you make italian pasta ")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [11.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Making Italian pasta from scratch is a rewarding process that involves just a few simple ingredients. Here's a basic recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups (250g) all-purpose flour (or \"00\" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### Step 1: Prepare the Dough\n1. **Make a Flour Well**: On a clean surface or in a large bowl, mound the flour and make a well in the center. Sprinkle a pinch of salt over the flour.\n  \n2. **Add Eggs**: Crack the eggs into the well. Use a fork to beat the eggs gently, then gradually incorporate the flour from the edges of the well.\n\n3. **Form the Dough**: As the mixture comes together, use your hands to knead it i

'Making Italian pasta from scratch is a rewarding process that involves just a few simple ingredients. Here\'s a basic recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups (250g) all-purpose flour (or "00" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### Step 1: Prepare the Dough\n1. **Make a Flour Well**: On a clean surface or in a large bowl, mound the flour and make a well in the center. Sprinkle a pinch of salt over the flour.\n  \n2. **Add Eggs**: Crack the eggs into the well. Use a fork to beat the eggs gently, then gradually incorporate the flour from the edges of the well.\n\n3. **Form the Dough**: As the mixture comes together, use your hands to knead it into a dough. If the dough is too dry, add a little water, a teaspoon at a time. If it\'s too sticky, sprinkle with a bit more flour.\n\n#### Step 2: Knead the Dough\n4. **Knead**: Knead the dough for about 8-10 minutes until it is smooth and ela

In [ ]:
chat.predict("How do you make italian pasta ")

In [48]:
set_debug(False)

# Cost Usage

In [50]:
with get_openai_callback() as usage:
    result = chat.predict("What is the recipe for meals using chicken breast")
    print(result)
    print(usage)

Here are three delicious recipes using chicken breast:

### 1. **Grilled Lemon Herb Chicken Breast**

#### Ingredients:
- 2 chicken breasts
- 2 tablespoons olive oil
- Juice of 1 lemon
- 2 teaspoons dried oregano
- 2 teaspoons garlic powder
- Salt and pepper to taste

#### Instructions:
1. **Marinate the Chicken**: In a bowl, mix olive oil, lemon juice, oregano, garlic powder, salt, and pepper. Add the chicken breasts and coat well. Let it marinate for at least 30 minutes (or up to 4 hours in the fridge).
  
2. **Preheat the Grill**: Preheat your grill to medium-high heat.

3. **Grill the Chicken**: Remove chicken from the marinade and grill for about 6-7 minutes on each side, or until the internal temperature reaches 165°F (75°C).

4. **Serve**: Let the chicken rest for a few minutes before slicing. Serve with a side of grilled vegetables or a fresh salad.

---

### 2. **Creamy Garlic Chicken Breast**

#### Ingredients:
- 2 chicken breasts
- 2 tablespoons olive oil
- 4 cloves garlic, 

In [58]:
chat = OpenAI(
    model="gpt-4o",
    max_tokens=450,
    temperature=0.1,
)

chat.save("model.json")

In [60]:
loaded_chat = load_llm("model.json")
loaded_chat

/Users/seongbae/miniconda3/envs/fullstack-gpt/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/seongbae/miniconda3/envs/fullstack-gpt/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4o', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})